# Notebook 08: Electronic Properties

## Calculate ONLY After Structure is Validated and Stable!

### Prerequisites Reminder

Before calculating electronic properties, ensure:

- ☑ Structure from reliable database
- ☑ Convergence testing complete
- ☑ Structure optimized
- ☑ **Stability verified** (thermodynamic, dynamic, mechanical)

**If ANY step is skipped → Results are MEANINGLESS!**

---

In [ ]:
# =============================================================================
# SETUP - Run this cell first (downloads pseudopotentials automatically)
# =============================================================================

import sys
from pathlib import Path
import importlib.util
import re

_cwd = Path.cwd().resolve()
if (_cwd / 'workshop_setup.py').exists():
    sys.path.insert(0, str(_cwd))
elif (_cwd / 'notebooks_enhanced' / 'workshop_setup.py').exists():
    sys.path.insert(0, str(_cwd / 'notebooks_enhanced'))
else:
    raise FileNotFoundError("Cannot find workshop_setup.py. Run from the workshop root or notebooks_enhanced/")

from workshop_setup import *

from ase import Atoms
from ase.build import bulk
from ase.io.espresso import write_espresso_in

SEEKPATH_AVAILABLE = (importlib.util.find_spec('seekpath') is not None)
if SEEKPATH_AVAILABLE:
    import seekpath
    print("✓ seekpath available - automatic k-path generation")
else:
    print("⚠ seekpath not available. Install: pip install seekpath")

# =============================================================================
# >>> STUDENT: MODIFY THIS SECTION <<<
# =============================================================================

MATERIAL = 'Si'
STRUCTURE_TYPE = 'diamond'
LATTICE_A = 5.43

FUNCTIONAL = 'PBE'
# FUNCTIONAL = 'LDA'
# FUNCTIONAL = 'PBEsol'

# =============================================================================
# AUTOMATIC SETUP
# =============================================================================

atoms = bulk(MATERIAL, STRUCTURE_TYPE, a=LATTICE_A)
elements = list(set(atoms.get_chemical_symbols()))

pp_files = setup_pseudopotentials(elements, functional=FUNCTIONAL)
ecutwfc, ecutrho = get_recommended_cutoffs(elements, functional=FUNCTIONAL)
PSEUDO_DIR_FUNC = get_pseudo_dir(FUNCTIONAL)

WORK_DIR = OUTPUT_DIR / '08_electronic'
WORK_DIR.mkdir(parents=True, exist_ok=True)
(WORK_DIR / 'tmp').mkdir(exist_ok=True)

print(f"\n{'='*70}")
print(f"ELECTRONIC PROPERTIES SETUP")
print(f"{'='*70}")
print(f"Material:   {MATERIAL} ({STRUCTURE_TYPE})")
print(f"Functional: {FUNCTIONAL}")
print(f"Cutoffs:    ecutwfc={ecutwfc} Ry")
print(f"{'='*70}")

---

## 1. Band Structure Calculation Workflow

```
Step 1: SCF calculation (dense k-mesh)
        ↓
Step 2: Bands calculation (k-path through BZ)
        ↓
Step 3: Post-processing with bands.x
        ↓
Step 4: Plot and analyze
```

In [ ]:
# =============================================================================
# AUTOMATIC K-PATH GENERATION USING SEEKPATH
# =============================================================================
# seekpath finds the correct high-symmetry path for ANY crystal structure.
# This is more reliable than manually specifying paths.
# =============================================================================

def get_kpath_from_structure(atoms):
    """
    Get high-symmetry k-path using seekpath.
    
    Parameters
    ----------
    atoms : ase.Atoms
        Structure
    
    Returns
    -------
    dict with path info, or None if seekpath unavailable
    """
    if not SEEKPATH_AVAILABLE:
        print("⚠ seekpath not available, using manual FCC path")
        return None
    
    # Convert ASE to seekpath format
    cell = atoms.get_cell().tolist()
    positions = atoms.get_scaled_positions().tolist()
    numbers = atoms.get_atomic_numbers().tolist()
    
    # Get path
    path_data = seekpath.get_path((cell, positions, numbers))
    
    return {
        'path': path_data['path'],
        'point_coords': path_data['point_coords'],
        'primitive_cell': path_data['primitive_lattice'],
    }


def generate_kpath_card_auto(atoms, npoints=20):
    """
    Generate K_POINTS card automatically using seekpath.
    """
    path_data = get_kpath_from_structure(atoms)
    
    if path_data is None:
        # Fallback to FCC path
        return HIGH_SYMMETRY_POINTS, STANDARD_PATHS['FCC']
    
    lines = ["K_POINTS {crystal_b}"]
    
    # Count segments
    n_segments = len(path_data['path'])
    lines.append(str(n_segments * 2))  # Start and end of each segment
    
    for i, (start, end) in enumerate(path_data['path']):
        start_coords = path_data['point_coords'][start]
        end_coords = path_data['point_coords'][end]
        
        # Add start point (with npoints to next)
        lines.append(f"  {start_coords[0]:.6f} {start_coords[1]:.6f} {start_coords[2]:.6f} {npoints}  ! {start}")
        # Add end point (0 points = just mark position)
        if i == len(path_data['path']) - 1:
            lines.append(f"  {end_coords[0]:.6f} {end_coords[1]:.6f} {end_coords[2]:.6f} 0  ! {end}")
    
    return '\n'.join(lines)


# High-symmetry points for manual fallback
HIGH_SYMMETRY_POINTS = {
    'FCC': {
        'G': (0.000, 0.000, 0.000),
        'X': (0.500, 0.000, 0.500),
        'W': (0.500, 0.250, 0.750),
        'K': (0.375, 0.375, 0.750),
        'L': (0.500, 0.500, 0.500),
    },
    'BCC': {
        'G': (0.000, 0.000, 0.000),
        'H': (0.500, -0.500, 0.500),
        'N': (0.000, 0.000, 0.500),
        'P': (0.250, 0.250, 0.250),
    },
}

STANDARD_PATHS = {
    'FCC': [('G', 20), ('X', 10), ('W', 10), ('K', 20), ('G', 20), ('L', 0)],
    'BCC': [('G', 20), ('H', 20), ('N', 20), ('G', 20), ('P', 0)],
}

# Test
print("K-path for current structure:")
print("=" * 50)
if SEEKPATH_AVAILABLE:
    path_data = get_kpath_from_structure(atoms)
    if path_data:
        print(f"Path segments: {path_data['path']}")
        print(f"\nHigh-symmetry points:")
        for name, coords in path_data['point_coords'].items():
            print(f"  {name}: ({coords[0]:.3f}, {coords[1]:.3f}, {coords[2]:.3f})")
else:
    print("Using manual FCC path (install seekpath for automatic detection)")

---

## 2. Bands Input File Generator

In [ ]:
# =============================================================================
# BAND STRUCTURE WITH AUTOMATIC PREREQUISITES
# =============================================================================
# This runs: SCF (if needed) → Bands → bands.x → Plot
# Students just call one function!
# =============================================================================

def run_band_structure_workflow(atoms, ecutwfc, prefix='bands', kpts_scf=(8,8,8)):
    """
    Run complete band structure calculation.
    
    Automatically runs SCF first if not already done.
    
    Parameters
    ----------
    atoms : ase.Atoms
        Structure
    ecutwfc : float
        Cutoff energy
    prefix : str
        Job prefix
    kpts_scf : tuple
        K-mesh for SCF
    
    Returns
    -------
    dict with band structure data
    """
    print("=" * 70)
    print("BAND STRUCTURE WORKFLOW")
    print("=" * 70)
    
    # Check if SCF exists, run if needed
    prereqs_met, missing = check_prerequisites(WORK_DIR, prefix, 'bands')
    
    if not prereqs_met:
        print(f"\n⚠ Prerequisites missing: {missing}")
        print("Running SCF calculation first...")
        
        scf_result = run_with_prerequisites(
            WORK_DIR, prefix, 'scf', atoms, ecutwfc, kpts_scf, 
            functional=FUNCTIONAL
        )
        
        if not scf_result.get('final', {}).get('converged', False):
            print("✗ SCF failed! Cannot proceed with bands.")
            return {'error': 'SCF failed'}
    else:
        print("✓ SCF already complete, proceeding to bands...")
    
    # Now run bands calculation
    elements = list(set(atoms.get_chemical_symbols()))
    pseudopotentials = {el: get_pseudopotential_filename(el, FUNCTIONAL) 
                       for el in elements}
    
    bands_file = WORK_DIR / f'{prefix}_bands.in'
    
    input_data = {
        'control': {
            'calculation': 'bands',
            'prefix': prefix,
            'outdir': './tmp',
            'pseudo_dir': str(PSEUDO_DIR_FUNC),
            'verbosity': 'high',
        },
        'system': {
            'ecutwfc': ecutwfc,
            'ecutrho': ecutwfc * 8,
            'occupations': 'smearing',
            'smearing': 'cold',
            'degauss': 0.01,
            'nbnd': len(atoms) * 4,
        },
        'electrons': {'conv_thr': 1.0e-8},
    }
    
    write_espresso_in(bands_file, atoms, input_data=input_data,
                      pseudopotentials=pseudopotentials, kpts=kpts_scf)
    
    # Replace K_POINTS with band path
    with open(bands_file, 'r') as f:
        content = f.read()
    
    # FCC path for diamond structure
    kpath_card = """K_POINTS {crystal_b}
6
  0.000000 0.000000 0.000000 20  ! G
  0.500000 0.000000 0.500000 10  ! X
  0.500000 0.250000 0.750000 10  ! W
  0.375000 0.375000 0.750000 20  ! K
  0.000000 0.000000 0.000000 20  ! G
  0.500000 0.500000 0.500000 0   ! L
"""
    content = re.sub(r'K_POINTS.*?(?=\n\n|\Z)', kpath_card, content, flags=re.DOTALL)
    with open(bands_file, 'w') as f:
        f.write(content)
    
    print("\nRunning bands calculation...")
    output, elapsed, _ = run_qe(bands_file, nprocs=NPROCS)
    
    # Post-processing with bands.x
    print("Running bands.x post-processing...")
    bandsx_file = WORK_DIR / f'{prefix}_bandsx.in'
    bandsx_input = f"""&BANDS
    prefix = '{prefix}'
    outdir = './tmp'
    filband = '{prefix}.bands.dat'
/
"""
    with open(bandsx_file, 'w') as f:
        f.write(bandsx_input)
    
    run_qe(bandsx_file, executable='bands.x', nprocs=1)
    
    print("\n" + "=" * 70)
    print("✓ Band structure complete!")
    print(f"  Output: {WORK_DIR / f'{prefix}.bands.dat.gnu'}")
    print("=" * 70)
    
    return {
        'bands_file': str(WORK_DIR / f'{prefix}.bands.dat.gnu'),
        'prefix': prefix,
    }


# To run (uncomment):
# result = run_band_structure_workflow(atoms, ecutwfc)

print("Band structure workflow function defined.")
print("\nUsage: result = run_band_structure_workflow(atoms, ecutwfc)")
print("This will automatically run SCF first if needed!")

---

## 3. Parsing Band Structure Output

In [ ]:
def parse_bands_gnu(filename: str) -> Tuple[np.ndarray, np.ndarray]:
    """
    Parse bands.dat.gnu file from QE bands.x.
    
    The GNU format has:
    - k-distance and energy for each band
    - Blank lines separate bands
    
    Returns
    -------
    k_distances : np.ndarray
        K-point distances along path
    bands : np.ndarray
        Band energies, shape (nkpts, nbands)
    """
    data = []
    current_band = []
    
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:  # Empty line = new band
                if current_band:
                    data.append(current_band)
                    current_band = []
            else:
                parts = line.split()
                if len(parts) >= 2:
                    k = float(parts[0])
                    e = float(parts[1])
                    current_band.append((k, e))
    
    if current_band:
        data.append(current_band)
    
    if not data:
        return None, None
    
    k_distances = np.array([p[0] for p in data[0]])
    bands = np.array([[p[1] for p in band] for band in data]).T
    
    return k_distances, bands

def plot_band_structure(k_distances: np.ndarray, bands: np.ndarray,
                        fermi_energy: float = 0.0,
                        high_sym_positions: List[float] = None,
                        high_sym_labels: List[str] = None,
                        title: str = 'Band Structure',
                        save_path: str = None):
    """
    Plot band structure with proper formatting.
    """
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Shift energies relative to Fermi level
    bands_shifted = bands - fermi_energy
    
    # Plot bands
    for i in range(bands.shape[1]):
        ax.plot(k_distances, bands_shifted[:, i], 'b-', linewidth=1.5)
    
    # Add Fermi level
    ax.axhline(y=0, color='r', linestyle='--', linewidth=1, label='$E_F$')
    
    # High-symmetry lines
    if high_sym_positions:
        for pos in high_sym_positions:
            ax.axvline(x=pos, color='gray', linestyle='--', alpha=0.5)
    
    # Labels
    if high_sym_positions and high_sym_labels:
        ax.set_xticks(high_sym_positions)
        ax.set_xticklabels(high_sym_labels, fontsize=14)
    
    ax.set_xlabel('Wave Vector', fontsize=14)
    ax.set_ylabel('Energy (eV)', fontsize=14)
    ax.set_title(title, fontsize=16)
    ax.set_xlim(k_distances.min(), k_distances.max())
    ax.grid(True, alpha=0.3, axis='y')
    ax.legend()
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
    
    plt.show()
    
    return fig, ax

print("Band structure plotting functions defined.")

---

## 3.5 Visualizing Charge Density - See the Bonds!

**This is DENSITY Functional Theory!** After band calculations, visualize where the electrons are.

For solids, we can visualize:
- **Total charge density** - Overall electron distribution
- **Partial charge density** - Electrons in specific energy ranges (like HOMO/LUMO in molecules!)
- **Band-decomposed charge** - See which orbitals contribute to which bands

In [ ]:
# =============================================================================
# CHARGE DENSITY VISUALIZATION - See the Electrons!
# =============================================================================
# After SCF/bands, generate cube files for VESTA visualization.
# This is analogous to HOMO/LUMO visualization in molecules!
# =============================================================================

def generate_charge_density_cube(prefix: str, work_dir, plot_type: str = 'charge'):
    """
    Generate charge density cube file using pp.x.
    
    Parameters
    ----------
    prefix : str
        QE calculation prefix
    work_dir : Path
        Working directory
    plot_type : str
        'charge' - Total charge density
        'partial' - Partial charge (specify energy range)
        'stm' - STM image (for surfaces)
    
    Returns
    -------
    Path to generated cube file
    """
    pp_file = work_dir / f'{prefix}_pp.in'
    cube_file = work_dir / f'{prefix}_{plot_type}.cube'
    
    plot_num = {'charge': 0, 'potential': 1, 'stm': 5, 'spin': 6, 'elf': 8}
    
    pp_input = f"""&INPUTPP
    prefix = '{prefix}'
    outdir = './tmp'
    filplot = '{prefix}.pp'
    plot_num = {plot_num.get(plot_type, 0)}
/

&PLOT
    nfile = 1
    filepp(1) = '{prefix}.pp'
    weight(1) = 1.0
    iflag = 3
    output_format = 6
    fileout = '{cube_file}'
/
"""
    
    with open(pp_file, 'w') as f:
        f.write(pp_input)
    
    print(f"Generating {plot_type} density cube file...")
    output, _, _ = run_qe(pp_file, executable='pp.x', nprocs=1)
    
    if cube_file.exists():
        print(f"✓ Saved: {cube_file}")
        return cube_file
    return None


def generate_partial_charge_cube(prefix: str, work_dir, 
                                  emin: float, emax: float,
                                  kpoint: int = 0, kband: int = 0):
    """
    Generate partial charge density for specific energy range.
    
    This is like visualizing HOMO/LUMO in molecules!
    - For valence band maximum: emin=-1, emax=0 (relative to Fermi)
    - For conduction band minimum: emin=0, emax=1
    
    Parameters
    ----------
    emin, emax : float
        Energy range relative to Fermi level (eV)
    kpoint, kband : int
        Specific k-point and band (0 = use energy window)
    """
    pp_file = work_dir / f'{prefix}_partial_pp.in'
    cube_file = work_dir / f'{prefix}_partial_{emin:.1f}_{emax:.1f}.cube'
    
    pp_input = f"""&INPUTPP
    prefix = '{prefix}'
    outdir = './tmp'
    filplot = '{prefix}_partial.pp'
    plot_num = 7
    emin = {emin}
    emax = {emax}
"""
    
    if kpoint > 0 and kband > 0:
        pp_input += f"""    kpoint = {kpoint}
    kband = {kband}
"""
    
    pp_input += f"""/

&PLOT
    nfile = 1
    filepp(1) = '{prefix}_partial.pp'
    weight(1) = 1.0
    iflag = 3
    output_format = 6
    fileout = '{cube_file}'
/
"""
    
    with open(pp_file, 'w') as f:
        f.write(pp_input)
    
    print(f"Generating partial charge ({emin} to {emax} eV)...")
    output, _, _ = run_qe(pp_file, executable='pp.x', nprocs=1)
    
    if cube_file.exists():
        print(f"✓ Saved: {cube_file}")
        return cube_file
    return None


# Show examples
print("=" * 70)
print("CHARGE DENSITY VISUALIZATION")
print("=" * 70)
print("""
After SCF calculation, visualize electron density:

1. TOTAL CHARGE DENSITY
   cube_file = generate_charge_density_cube(prefix, WORK_DIR, 'charge')
   → Shows overall bonding character
   
2. VALENCE BAND (like HOMO in molecules)
   cube_file = generate_partial_charge_cube(prefix, WORK_DIR, emin=-2, emax=0)
   → Shows highest occupied states
   
3. CONDUCTION BAND (like LUMO in molecules)
   cube_file = generate_partial_charge_cube(prefix, WORK_DIR, emin=0, emax=2)
   → Shows lowest unoccupied states (needs NSCF with empty bands)

VIEWING IN VESTA:
-----------------
1. Open VESTA → File → Open → Select .cube file
2. Properties → Isosurfaces → Set level (try 0.01-0.1 e/Å³)
3. For bonding: Look for charge accumulation between atoms
4. For antibonding: Look for nodal planes

What to look for:
- Si: Charge concentrated along bonds (covalent)
- NaCl: Charge on Cl, depleted on Na (ionic)
- Metals: Uniform distribution (metallic)
""")

---

## 4. Density of States (DOS)

### DOS Workflow

```
Step 1: SCF calculation (same as before)
        ↓
Step 2: NSCF with VERY dense k-mesh
        ↓
Step 3: dos.x for total DOS
        ↓
Step 4: projwfc.x for projected DOS (PDOS)
```

In [ ]:
def generate_nscf_input(prefix: str, ecutwfc: float, ecutrho: float,
                        pseudo_dir: str, kpoints: Tuple[int, int, int],
                        cell_parameters, atomic_species, atomic_positions) -> str:
    """
    Generate NSCF input for DOS calculation.
    
    Note: K-mesh should be MUCH denser than SCF (e.g., 16×16×16 or more)
    """
    nat = len(atomic_positions)
    ntyp = len(atomic_species)
    kx, ky, kz = kpoints
    
    input_text = f"""&CONTROL
    calculation = 'nscf'
    prefix = '{prefix}'
    outdir = './tmp'
    pseudo_dir = '{pseudo_dir}'
    verbosity = 'high'
/

&SYSTEM
    ibrav = 0
    nat = {nat}
    ntyp = {ntyp}
    ecutwfc = {ecutwfc}
    ecutrho = {ecutrho}
    occupations = 'tetrahedra'
/

&ELECTRONS
    conv_thr = 1.0e-8
/

ATOMIC_SPECIES
"""
    
    for symbol, mass, pp_file in atomic_species:
        input_text += f"    {symbol}  {mass}  {pp_file}\n"
    
    input_text += "\nCELL_PARAMETERS {angstrom}\n"
    for vec in cell_parameters:
        input_text += f"    {vec[0]:12.8f}  {vec[1]:12.8f}  {vec[2]:12.8f}\n"
    
    input_text += "\nATOMIC_POSITIONS {crystal}\n"
    for symbol, x, y, z in atomic_positions:
        input_text += f"    {symbol}  {x:12.8f}  {y:12.8f}  {z:12.8f}\n"
    
    input_text += f"\nK_POINTS {{automatic}}\n    {kx} {ky} {kz} 0 0 0\n"
    
    return input_text

def generate_dos_input(prefix: str, emin: float = -15.0, emax: float = 15.0,
                       deltae: float = 0.01, fildos: str = 'dos.dat') -> str:
    """
    Generate dos.x input file.
    """
    return f"""&DOS
    prefix = '{prefix}'
    outdir = './tmp'
    fildos = '{fildos}'
    Emin = {emin}
    Emax = {emax}
    DeltaE = {deltae}
/
"""

def parse_dos_output(filename: str) -> Tuple[np.ndarray, np.ndarray, np.ndarray, float]:
    """
    Parse DOS output file from dos.x.
    
    Returns
    -------
    energy : np.ndarray
    dos : np.ndarray
    idos : np.ndarray (integrated DOS)
    fermi : float (Fermi energy)
    """
    energy = []
    dos = []
    idos = []
    fermi = None
    
    with open(filename, 'r') as f:
        for line in f:
            if line.startswith('#'):
                match = re.search(r'EFermi\s*=\s*([\d.+-]+)', line)
                if match:
                    fermi = float(match.group(1))
                continue
            
            parts = line.split()
            if len(parts) >= 2:
                energy.append(float(parts[0]))
                dos.append(float(parts[1]))
                if len(parts) >= 3:
                    idos.append(float(parts[2]))
    
    return np.array(energy), np.array(dos), np.array(idos) if idos else None, fermi

print("DOS calculation functions defined.")
print("\nExample dos.x input:")
print(generate_dos_input('silicon'))

---

## 5. Band Gap Problem in DFT

### Why DFT Underestimates Band Gaps

| Material | DFT (PBE) | Experiment | Error |
|----------|-----------|------------|-------|
| Si | ~0.5 eV | 1.17 eV | -57% |
| Ge | ~0.0 eV | 0.74 eV | -100% |
| GaAs | ~0.5 eV | 1.52 eV | -67% |
| MgO | ~4.5 eV | 7.8 eV | -42% |

### Solutions

1. **GW calculations**: Most accurate, but expensive
2. **Hybrid functionals (HSE06)**: Good balance of accuracy and cost
3. **Scissors operator**: Empirical shift (quick and dirty)
4. **DFT+U**: For d/f electron systems

---

## 6. Effective Mass Calculation

In [ ]:
# =============================================================================
# EFFECTIVE MASS CALCULATION - Proper Implementation
# =============================================================================
# Effective mass is crucial for transport properties and device modeling.
# m* = ℏ² / (d²E/dk²)
#
# For accurate results:
# - Use VERY dense k-mesh near band extrema
# - Fit parabola to multiple points (not just 3-point finite difference)
# - Consider anisotropy (different masses along different directions)
# =============================================================================

def calculate_effective_mass_parabolic(k_values: np.ndarray, 
                                        energies: np.ndarray,
                                        fit_range: int = 5) -> dict:
    """
    Calculate effective mass by parabolic fitting near band extremum.
    
    This is MORE RELIABLE than simple finite difference!
    
    Parameters
    ----------
    k_values : np.ndarray
        K-point values (in 1/Å or 2π/a units)
    energies : np.ndarray
        Band energies in eV
    fit_range : int
        Number of points on each side of extremum to use for fitting
    
    Returns
    -------
    dict with effective mass and fit quality
    """
    # Find band extremum (minimum for electrons, maximum for holes)
    idx_min = np.argmin(energies)
    idx_max = np.argmax(energies)
    
    results = {}
    
    # Electron effective mass (near CBM)
    if idx_min > fit_range and idx_min < len(k_values) - fit_range:
        k_fit = k_values[idx_min - fit_range:idx_min + fit_range + 1]
        e_fit = energies[idx_min - fit_range:idx_min + fit_range + 1]
        
        # Shift to center at minimum
        k_centered = k_fit - k_values[idx_min]
        e_centered = e_fit - energies[idx_min]
        
        # Fit parabola: E = E0 + (ℏ²k²)/(2m*)
        # => E = a*k² => m* = ℏ²/(2a)
        coeffs = np.polyfit(k_centered, e_centered, 2)
        a = coeffs[0]  # Coefficient of k²
        
        # ℏ² in eV·Å²·m_e units = 7.6199
        HBAR_SQ = 7.6199  # eV·Å²·m_e
        
        if abs(a) > 1e-10:
            m_star_e = HBAR_SQ / (2 * a)
            results['electron'] = {
                'm_star': abs(m_star_e),
                'fit_coeff': a,
                'k_min': k_values[idx_min],
                'E_min': energies[idx_min],
            }
    
    # Hole effective mass (near VBM) - note: curvature is negative
    if idx_max > fit_range and idx_max < len(k_values) - fit_range:
        k_fit = k_values[idx_max - fit_range:idx_max + fit_range + 1]
        e_fit = energies[idx_max - fit_range:idx_max + fit_range + 1]
        
        k_centered = k_fit - k_values[idx_max]
        e_centered = e_fit - energies[idx_max]
        
        coeffs = np.polyfit(k_centered, e_centered, 2)
        a = coeffs[0]
        
        HBAR_SQ = 7.6199
        
        if abs(a) > 1e-10:
            m_star_h = HBAR_SQ / (2 * abs(a))
            results['hole'] = {
                'm_star': m_star_h,
                'fit_coeff': a,
                'k_max': k_values[idx_max],
                'E_max': energies[idx_max],
            }
    
    return results


def calculate_dos_effective_mass(m_l: float, m_t: float, 
                                  valley_degeneracy: int = 1) -> float:
    """
    Calculate density-of-states effective mass from anisotropic masses.
    
    For ellipsoidal valleys (like Si conduction band):
    m_dos = N_v^(2/3) * (m_l * m_t^2)^(1/3)
    
    Parameters
    ----------
    m_l : float
        Longitudinal effective mass (along valley axis)
    m_t : float
        Transverse effective mass (perpendicular to valley)
    valley_degeneracy : int
        Number of equivalent valleys (6 for Si CB, 1 for Ge CB)
    
    Returns
    -------
    float : DOS effective mass in m_0 units
    """
    m_dos = valley_degeneracy**(2/3) * (m_l * m_t**2)**(1/3)
    return m_dos


# Example: Silicon effective masses (literature values for comparison)
print("=" * 70)
print("EFFECTIVE MASS CALCULATION")
print("=" * 70)

print("""
Silicon effective masses (for reference):

ELECTRONS (conduction band - 6 equivalent valleys):
  Longitudinal:  m_l = 0.98 m₀
  Transverse:    m_t = 0.19 m₀
  DOS effective: m_dos = 1.08 m₀ (including valley degeneracy)

HOLES (valence band - at Γ point):
  Heavy hole:    m_hh = 0.49 m₀
  Light hole:    m_lh = 0.16 m₀
  Split-off:     m_so = 0.29 m₀

To calculate from YOUR band structure:
--------------------------------------
1. Run bands calculation with DENSE k-mesh near extrema
2. Parse band energies: k_vals, bands = parse_bands_gnu('bands.dat.gnu')
3. For each band of interest:
   result = calculate_effective_mass_parabolic(k_vals, bands[:, band_idx])
   print(f"m* = {result['electron']['m_star']:.3f} m₀")

Note: DFT typically gives ~10-20% error in effective masses
""")

# Calculate DOS effective mass for Si
m_dos_Si = calculate_dos_effective_mass(m_l=0.98, m_t=0.19, valley_degeneracy=6)
print(f"Si DOS effective mass (electrons): {m_dos_Si:.2f} m₀")

---

## Summary

### Electronic Property Calculations

| Property | Method | Key Parameters |
|----------|--------|----------------|
| Band structure | SCF → bands → bands.x | K-path, nbnd |
| DOS | SCF → NSCF → dos.x | Dense k-mesh, tetrahedra |
| PDOS | ... → projwfc.x | Orbital projections |
| Effective mass | From band curvature | d²E/dk² |

### Key Reminders

1. **ALWAYS verify stability first**
2. **DFT underestimates band gaps** - don't trust quantitatively
3. **Use dense k-mesh for DOS** - at least 2× the SCF grid
4. **Qualitative trends are reliable** - direct vs indirect, orbital character

### Next Notebook
→ **09_Advanced_Properties.ipynb**: Optical, transport, and other properties